In [1]:
from qmenta.core.platform import Auth, post, parse_response
from getpass import getpass
import pandas as pd
import numpy as np
import utils
import utils_cleaning
import forms
import datetime

import sys
sys.path.append("../..")

from MSDA_Querry2 import querries

## Registries

In [3]:
project_id_reg = 3202

# base url to connect to the central platform
base_url = "https://platform.qmenta.com"
# PUT YOUR USERNAME (EMAIL) HERE
username = "edward.debrouwer@esat.kuleuven.be"
# you will be asked for your password here
password = getpass()

# creation of authentication object
auth_obj = Auth.login(username, password, base_url)

# method to fetch the subjects data
def get_subjects_data(project_id):
    r = post(auth_obj, "/patient_manager/get_patient_list",
            {"_pid":project_id},
            timeout=600.0)

    data_trans = [{
            "id": record["_id"],
            "secret_name":record["patient_secret_name"],
            **{
                k[3:]:record[k]
                for k in record
                if k[:3] == "md_"
            }
    } for record in parse_response(r)]

    for r in data_trans:
        for k in r:
            if isinstance(r[k], dict):
                try:
                    r[k] = datetime.datetime.fromtimestamp(r[k]["$date"]/1000.0)
                except:
                    r[k] = None

    return data_trans

data_reg = get_subjects_data(project_id_reg)
df = pd.DataFrame(data_reg)



········


### Enhance

In [4]:
print("Cleaning data ....")
df_reg = utils_cleaning.clean_data(df,auth_obj,project_id_reg, None, send_qa_staus=False) # Set to true to update qa status

Cleaning data ....
Unable to clean year_reporting as the column is not in the dataframe


In [5]:
df_reg = utils.enhance_registry_data(df_reg)
df_reg.head()

,id,secret_name,age_years,com_cardiovascular_disease,com_chronic_kidney_disease,com_diabetes,com_hypertension,com_lung_disease,com_malignancy,covid19_admission_hospital,...,year_onset,year_reporting,disease_duration,disease_duration_in_cat,disease_duration_in_cat2,ms_type2,current_or_former_smoker,type_last_dmt,type_current_dmt,dmt_type_overall
0,1043987.0,COV0_1,35.0,None,None,None,None,None,None,no,...,2004.0,2020.0,16.0,2,1,relapsing-remitting,None,None,fingolimod,currently on fingolimod
1,1043996.0,COV0_10,43.0,None,None,None,None,None,None,no,...,2004.0,2020.0,16.0,2,1,relapsing-remitting,None,fingolimod,None,currently not using any DMT
2,1043997.0,COV0_11,31.0,None,None,None,None,None,None,no,...,2020.0,2020.0,0.0,None,None,relapsing-remitting,None,None,rituximab,currently on rituximab
3,1043998.0,COV0_12,64.0,None,None,None,None,None,None,yes,...,1994.0,2020.0,26.0,2,1,progressive MS,yes,None,rituximab,currently on rituximab
4,1043999.0,COV0_13,31.0,None,None,None,None,None,None,no,...,2017.0,2020.0,3.0,1,0,relapsing-remitting,None,None,rituximab,currently on rituximab


## Forms

In [6]:
project_id_forms = 3150
data_forms = get_subjects_data(project_id_forms)
df = pd.DataFrame(data_forms)


In [7]:
print("Cleaning data ....")
df_forms = utils_cleaning.clean_data(df,auth_obj,project_id_forms, None, send_qa_staus=False)

Cleaning data ....
Unable to clean year_reporting as the column is not in the dataframe


In [8]:
df_forms = utils.enhance_forms_data(df_forms)
df_forms.tail()

,id,secret_name,covid19_date_reporting,covid19_has_symptoms,covid19_sympt_fever,covid19_sympt_dry_cough,covid19_sympt_fatigue,covid19_sympt_pain,covid19_sympt_sore_throat,covid19_sympt_shortness_breath,...,year_onset,year_reporting,disease_duration,disease_duration_in_cat,disease_duration_in_cat2,ms_type2,current_or_former_smoker,type_last_dmt,type_current_dmt,dmt_type_overall
1249,1013318.0,P_993,NaT,yes,no,no,no,no,yes,no,...,2005.0,NaN,NaN,None,None,relapsing-remitting,None,None,fingolimod,currently on another drug not listed
1250,1013406.0,P_994,NaT,no,,,,,,,...,2013.0,NaN,NaN,None,None,relapsing-remitting,yes,None,ocrelizumab,currently on another drug not listed
1251,1013410.0,P_995,NaT,no,,,,,,,...,2018.0,NaN,NaN,None,None,relapsing-remitting,None,None,ocrelizumab,currently on another drug not listed
1252,1013421.0,P_998,NaT,no,,,,,,,...,2015.0,NaN,NaN,None,None,relapsing-remitting,None,None,dimethyl_fumarate,currently on another drug not listed
1253,1013422.0,P_999,NaT,no,,,,,,,...,2015.0,NaN,NaN,None,None,,None,None,natalizumab,currently on another drug not listed


In [9]:
merged_df = pd.concat([df_forms, df_reg])

In [10]:
querries.compute_tables(merged_df[merged_df["report_source"]=="patients"], report_source = "patients")

KeyboardInterrupt: 

In [ ]:
querries.compute_tables(merged_df[merged_df["report_source"]=="clinicians"], report_source = "clinicians")